In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 days 22 hours 37 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_siyushen_bgs1bh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.635 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [3]:
train = h2o.import_file("https://raw.githubusercontent.com/rebeccaisnotabug/autoML/master/train.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train.head()

revenue,outcalls,incalls,months,eqpdays,webcap,marryyes,travel,pcown,creditcd,retcalls,churndep
83.53,20,1,31,745,1,0,0,0,0,4,1
29.99,0,0,52,1441,0,0,0,1,1,3,1
51.42,0,0,36,59,1,0,0,0,0,4,1
37.75,2.67,0,25,572,0,0,0,1,1,3,1
5,0,0,26,785,1,0,0,0,1,3,1
42.71,8.67,0,27,224,1,0,0,0,0,3,1
33.66,8.33,0,31,933,1,0,0,0,0,2,1
52.56,80,31.67,33,402,1,0,0,0,1,3,1
98.47,24.67,3.33,35,13,0,0,1,1,1,3,1
7.82,0,0,24,561,0,0,0,0,0,2,1


In [5]:
test = h2o.import_file("https://raw.githubusercontent.com/rebeccaisnotabug/autoML/master/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
test.head()

revenue,outcalls,incalls,months,eqpdays,webcap,marryyes,travel,pcown,creditcd,retcalls,churndep
48.82,10,3,26,780,0,0,0,0,1,4,1
5.25,0,0,45,1354,0,0,0,0,0,2,1
26.84,4,1.33,49,1471,0,1,0,1,1,2,1
53.69,15,2.33,23,267,1,0,0,0,1,3,1
22.5,8.67,2.67,37,243,1,0,0,0,0,3,1
49.16,8.67,0,47,1434,0,1,0,0,1,1,1
69.11,18,2.67,48,1430,0,0,0,0,1,1,1
15,1,0.33,14,408,1,0,0,0,0,2,1
61.72,19.33,39,16,464,1,0,0,0,0,2,1
58.27,4.33,0.67,37,1138,0,0,0,0,1,1,1


In [7]:
x = train.columns
y = "churndep"
x.remove(y)

In [8]:
train[y] = train[y].asfactor()

In [9]:
aml = H2OAutoML()

In [10]:
aml.train(x=x, y=y, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
lb = aml.leaderboard
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20210524_143600,0.637468,0.662391,0.61519,0.44795,0.484808,0.235038
StackedEnsemble_AllModels_AutoML_20210524_143600,0.63666,0.662746,0.61362,0.436003,0.48497,0.235196
XGBoost_grid__1_AutoML_20210524_143600_model_2,0.636098,0.662882,0.61344,0.439803,0.485054,0.235277
GBM_grid__1_AutoML_20210524_143600_model_2,0.635988,0.663427,0.614355,0.434975,0.485294,0.235511
GBM_1_AutoML_20210524_143600,0.635442,0.663882,0.609335,0.426443,0.48547,0.235681
GBM_grid__1_AutoML_20210524_143600_model_9,0.63336,0.663966,0.611472,0.433453,0.485566,0.235775
XGBoost_grid__1_AutoML_20210524_143600_model_11,0.632746,0.665288,0.610233,0.431771,0.486131,0.236323
GBM_grid__1_AutoML_20210524_143600_model_13,0.632674,0.664731,0.610623,0.429262,0.485924,0.236122
GBM_2_AutoML_20210524_143600,0.632599,0.664524,0.609116,0.440657,0.485824,0.236025
XGBoost_grid__1_AutoML_20210524_143600_model_12,0.631906,0.66548,0.608005,0.441408,0.486236,0.236426


In [12]:
pred = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [14]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.23571023095316987
RMSE: 0.48549998038431463
LogLoss: 0.6637843228855111
Null degrees of freedom: 12100
Residual degrees of freedom: 12094
Null deviance: 16775.866242603166
Residual deviance: 16064.908182475141
AIC: 16078.908182475141
AUC: 0.6347936165313285
AUCPR: 0.6142269699581044
Gini: 0.269587233062657

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3142727933372383: 


,,0,1,Error,Rate
0,0,913.0,5122.0,0.8487,(5122.0/6035.0)
1,1,309.0,5757.0,0.0509,(309.0/6066.0)
2,Total,1222.0,10879.0,0.4488,(5431.0/12101.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.314273,0.679492,340.0
1,max f2,0.227271,0.834434,396.0
2,max f0point5,0.502450,0.605475,207.0
3,max accuracy,0.502450,0.602182,207.0
4,max precision,0.847513,1.000000,0.0
5,max recall,0.227271,1.000000,396.0
6,max specificity,0.847513,1.000000,0.0
7,max absolute_mcc,0.502450,0.206275,207.0
8,max min_per_class_accuracy,0.522222,0.596769,188.0
9,max mean_per_class_accuracy,0.502450,0.601992,207.0



Gains/Lift Table: Avg response rate: 50.13 %, avg score: 49.82 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010082,0.747619,1.520694,1.520694,0.762295,0.779858,0.762295,0.779858,0.015331,0.015331,52.069449,52.069449,0.010526
1,2,0.020081,0.718128,1.368395,1.444858,0.685950,0.732355,0.724280,0.756204,0.013683,0.029014,36.839531,44.485827,0.017912
2,3,0.030080,0.698619,1.368395,1.419441,0.685950,0.706937,0.711538,0.739827,0.013683,0.042697,36.839531,41.944064,0.025298
3,4,0.040079,0.687802,1.500289,1.439611,0.752066,0.692988,0.721649,0.728141,0.015002,0.057699,50.028883,43.961101,0.035329
4,5,0.050079,0.678987,1.269475,1.405640,0.636364,0.683606,0.704620,0.719249,0.012694,0.070392,26.947517,40.564000,0.040732
5,6,0.100074,0.645592,1.282665,1.344203,0.642975,0.661062,0.673823,0.690179,0.064128,0.134520,28.266452,34.420303,0.069069
6,7,0.150070,0.620998,1.282665,1.323701,0.642975,0.633002,0.663546,0.671131,0.064128,0.198648,28.266452,32.370149,0.097405
7,8,0.200066,0.597634,1.196934,1.292023,0.600000,0.609286,0.647666,0.655676,0.059842,0.258490,19.693373,29.202264,0.117148
8,9,0.300058,0.565924,1.183744,1.255940,0.593388,0.580214,0.629579,0.630529,0.118365,0.376855,18.374438,25.593983,0.153988
9,10,0.400050,0.543656,1.069986,1.209461,0.536364,0.554272,0.606280,0.611469,0.106990,0.483844,6.998621,20.946103,0.168020
